In [1]:
from pyomo.environ import *
from pyomo.dae import *
import numpy as np

m = ConcreteModel()

t_end = 2*60*60
m.t = ContinuousSet(bounds=(0,t_end)) 

m.x = Var(m.t, bounds=(0,600e3))
m.v = Var(m.t, bounds=(0,150))
m.u = Var(m.t, initialize=0, bounds=(-200e3,200e3))

Mass = 300.0e3
v_max = 100.0
p_max = 8.0e6
f_max = p_max/v_max
x_end = 500e3

K_d = f_max/v_max**2
print('K_d: {:2.3f}'.format(K_d))

m.dx = DerivativeVar(m.x)
m.dv = DerivativeVar(m.v)

m.obj = Objective(expr=(m.x[t_end]-x_end)**2+m.v[t_end]**2)

def _obj(M):
    #e=0.0
    #x=0
    #for i in range(len(M.u)):

    #    e += M.u[i]**2
    #for i in range(len(M.x2)):
    #    x += (M.x2[i]-3)**2
    #e = Integral(expr=M.u, wrt=M.u, bounds=(0,1))
    #x = Integral(expr=(m.x2-2)**2, wrt=M.x1, bounds=(0,1))
    e = 0
    for i in M.t:
        p = m.v[i]*M.u[i]
        #if p<0:
        #    p=0
            
        e += p
        
    return 10*e + (m.x[t_end]-x_end)**2 + 100000*m.v[t_end]**2
m.obj = Objective(rule=_obj, sense=minimize, doc='Define objective function')

def _dx(M,i):
    if i == 0:
        return Constraint.Skip
    return M.dx[i] == M.v[i]

m.dxcon = Constraint(m.t, rule=_dx)

def _dv(M,i):
    if i == 0:
        return Constraint.Skip
    return M.dv[i] == 1/Mass*(M.u[i]  - 2*K_d*M.v[i]*abs(M.v[i]))
m.dvcon = Constraint(m.t, rule=_dv)


#def _power(M,i):
#    return M.u[i]*M.v[i] < 20.0e6
#m.power = Constraint(m.t, rule=_power)

def _init(M):
    yield M.x[0] == 0
    yield M.v[0] == 0
    yield ConstraintList.End
m.init_conditions = ConstraintList(rule=_init)

from pyomo.environ import *
from pyomo.dae import *
from pyomo.opt import SolverFactory
from pyomo.dae.plugins.finitedifference import Finite_Difference_Transformation
from pyomo.dae.plugins.colloc import Collocation_Discretization_Transformation


instance = m

# Discretize model using Backward Finite Difference method
#discretize = Finite_Difference_Transformation()
#disc_instance = discretize.apply(instance,nfe=20,scheme='BACKWARD')

# Discretize model using Orthogonal Collocation
discretize = Collocation_Discretization_Transformation()
disc_instance = discretize.apply(instance,nfe=40,ncp=10,scheme='LAGRANGE-RADAU')

# Will reimplement this method in future release of pyomo.dae
# disc_instance = discretize.reduce_collocation_points(var=instance.u,
# 	ncp=2, diffset=instance.t)

solver='ipopt'
opt=SolverFactory(solver)

results = opt.solve(disc_instance,tee=True)
disc_instance.load(results)

x = []
v = []
u = []
p = []
t=[]

#print(sorted(disc_instance.t))

for i in sorted(disc_instance.t):
    t.append(i)
    x.append(value(disc_instance.x[i]))
    v.append(value(disc_instance.v[i]))
    u.append(value(disc_instance.u[i]))
    p.append(value(disc_instance.u[i])*value(disc_instance.v[i]))

t = np.array(t)    
x = np.array(x)    
v = np.array(v)    
u = np.array(u)    

K_d: 8.000


TypeError: 'dict_keys' object does not support indexing

In [5]:
import matplotlib.pyplot as plt


p = v*u

fig, (ax0,ax1) = plt.subplots(nrows=2)   # creates a figure with one axe

ax0.plot(t,x/1.0e3)
ax1.plot(t,v, 'g', lw=1)
#ax1.plot(t,v_1)
#ax1.plot(t,v_2)
#ax1.plot(t,v_3)

fig.show()

fig, (ax0,ax1) = plt.subplots(nrows=2)   # creates a figure with one axe

ax0.plot(t,u/1.0e3) 
ax1.plot(t,p/1.0e6) 
plt.show()
